In [4]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

This file creates the trade file behind the [Phase One Tracker](https://www.tradewartracker.com/). It proceeds in several steps.

1. Grabs the trade data

2. Constructs the relavent Phase One product catagories and the associated goals/targets.

3. Maps the data to the county level.

In [5]:
trade_type = "exports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

---
#### Grabe the trade data using the Census's API

In [6]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["I_COMMODITY"].astype(str)
        
        df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["E_COMMODITY"].astype(str)
        
        df["china_exports_quant"] = df["QTY_1_MO"].astype(float)
    
        df["quant_type"] = df["UNIT_QY1"].astype(str)
        
        df.drop(["ALL_VAL_MO", "E_COMMODITY", "COMM_LVL", "QTY_1_MO", "UNIT_QY1"], axis = 1, inplace = True)
    
    return df

In [91]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO,UNIT_QY1,QTY_1_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS6"

url = url + "&CTY_CODE=5700"

df = census_trade(url, trade_type, "china", "hs6")

df["hs4"] = df["hs6"].str[0:4]

df.time.max()

<Response [200]>


Timestamp('2020-05-01 00:00:00')

In [92]:
df.head()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4
0,2013-01-01,5700,287000.0,481031,0.0,-,4810
1,2013-01-01,5700,311650.0,481160,0.0,-,4811
2,2013-01-01,5700,63701.0,481620,0.0,-,4816
3,2013-01-01,5700,40265.0,490290,0.0,-,4902
4,2013-01-01,5700,324939.0,480431,0.0,-,4804


---

Now this is the **new** stuff from the Census. The plan is to append it to the regular data set...

In [93]:
census_yrl = "https://www.census.gov/foreign-trade/Press-Release/2020pr/06/Prelim_US_C_ETA/June%202020.xlsx"

dfC = pd.read_excel(census_yrl, skiprows = [0,1])

dfC.columns = ["hs4_o", "C_description", "china_exports"]

dfC["time"] = "2020-06-01"

dfC.time = pd.to_datetime(dfC.time, format="%Y-%m-%d")

dfC.dropna(inplace = True)

dfC.head()

,hs4_o,C_description,china_exports,time
0,0101,"Horses, Asses, Mules And Hinnies, Live",0.0,2020-06-01
1,0102,"Bovine Animals, Live",0.0,2020-06-01
2,0103,"Swine, Live",28224.0,2020-06-01
3,0104,"Sheep And Goats, Live",0.0,2020-06-01
4,0105,"Poultry, Live; Chickens, Ducks, Geese, Turkeys...",769992.0,2020-06-01


In [94]:
dfC = dfC[["hs4_o","china_exports","time"]].merge(dfproducts, left_on = "hs4_o", right_on = "hs4_o", how = "left")

In [95]:
dfC.head()

,hs4_o,china_exports,time,description,low_catagory,high_catagory,hs4
0,0101,0.0,2020-06-01,"Hors,es ass,es mules and hinnies, live",Other agricultural commodities,2. Agriculture,0101
1,0102,0.0,2020-06-01,"Bovine anima,ls live",Other agricultural commodities,2. Agriculture,0102
2,0103,28224.0,2020-06-01,"Swine, live",Other agricultural commodities,2. Agriculture,0103
3,0104,0.0,2020-06-01,"Sheep and goa,ts live",Other agricultural commodities,2. Agriculture,0104
4,0105,769992.0,2020-06-01,"Poultry, live; chickens, ducks, geese, turkeys...",Other agricultural commodities,2. Agriculture,0105


---
#### Bring in the Phase One Product list

In [96]:
dfproducts = pd.read_csv(".\\data"+ "\\annex-6-1.csv", dtype = {"hs4": str, "hs4_o": str})

not_energy = dfproducts.high_catagory != "3. Energy"

energy_list = list(dfproducts.loc[~not_energy,"hs4"])

print(energy_list)

['2701', '2709', '2710', '2711', '2711', '2711', '2711', '2713', '2713', '2905']


Next step, we will remove the energy products, these will be added on below from the ``phase-one-energy`` notebook

In [97]:
df = df[df["hs4"].isin(energy_list) == False]

Now merge the non energy products with the product lists

In [98]:
df_phaseone = df.merge(dfproducts, left_on = "hs4", right_on = "hs4", how = "left", indicator = True)

Grabe the column names so the energy data will be the same order otherwise pandas complains

In [99]:
cnames = list(df_phaseone.columns)
print(cnames)

['time', 'CTY_CODE', 'china_exports', 'hs6', 'china_exports_quant', 'quant_type', 'hs4', 'hs4_o', 'description', 'low_catagory', 'high_catagory', '_merge']


Then read in the energy data set

In [100]:
file = ".\\data"+ "\\phaseone-energy.parquet"

df_energy = pq.read_table(file).to_pandas()

df_energy[df_energy.low_catagory == "Liquefied natural gas"]

,time,CTY_CODE,hs4_o,hs6,china_exports,china_exports_quant,quant_type,description,low_catagory,high_catagory,hs4,_merge
530,2013-04-01,5700,271111,271111,119031.0,996.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
531,2013-05-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
532,2013-06-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
533,2013-07-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
534,2013-08-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
...,...,...,...,...,...,...,...,...,...,...,...,...
590,2019-11-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
591,2019-12-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
592,2020-03-01,5700,271111,271111,58741925.0,641816.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
593,2020-04-01,5700,271111,271111,102792160.0,1131607.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both


And then append it to the main dataframe

In [101]:
df_phaseone = df_phaseone.append(df_energy)

In [102]:
df_phaseone = df_phaseone.append(dfC)

In [103]:
df_phaseone.tail()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
549,2020-06-01,NaN,259974022.0,NaN,NaN,NaN,9018,9018,"Instruments and appliances used in medic,al su...",Optical and medical instruments,1. Manufactured Goods,NaN
550,2020-06-01,NaN,7874237.0,NaN,NaN,NaN,9019,9019,Mechano-therapy appliances; massage apparatus;...,Optical and medical instruments,1. Manufactured Goods,NaN
551,2020-06-01,NaN,903873.0,NaN,NaN,NaN,9020,9020,"Other breathing appliances and gas masks, excl...",Optical and medical instruments,1. Manufactured Goods,NaN
552,2020-06-01,NaN,53251450.0,NaN,NaN,NaN,9021,9021,"Orthopedic appliances, including crutch,es su...",Optical and medical instruments,1. Manufactured Goods,NaN
553,2020-06-01,NaN,43611153.0,NaN,NaN,NaN,9022,9022,Apparatus based on the use of X-rays or of alp...,Optical and medical instruments,1. Manufactured Goods,NaN


In [105]:
# then we can see the stuff with the orginal code
df_phaseone[df_phaseone.hs4_o == "271112"]

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
595,2013-10-01,5700,49461021.0,271112,1105106.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
596,2013-11-01,5700,20872845.0,271112,388600.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
597,2013-12-01,5700,70036260.0,271112,1154000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
598,2014-02-01,5700,39743760.0,271112,577000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
599,2014-03-01,5700,29323140.0,271112,577000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
...,...,...,...,...,...,...,...,...,...,...,...,...
668,2019-12-01,5700,0.0,271112,0.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
669,2020-03-01,5700,47660255.0,271112,404018.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
670,2020-04-01,5700,46750976.0,271112,451670.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
671,2020-05-01,5700,56486683.0,271112,470767.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both


In [106]:
df_phaseone.high_catagory = df_phaseone.high_catagory.fillna("not in aggreement")

Then the outfiles ``phaseone-tradedata.parquet`` is the main file used in ``phase-one-plots.ipynb`` notebook

In [107]:
df_phaseone.head()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
0,2013-01-01,5700,287000.0,481031,0.0,-,4810,NaN,NaN,NaN,not in aggreement,left_only
1,2013-01-01,5700,311650.0,481160,0.0,-,4811,NaN,NaN,NaN,not in aggreement,left_only
2,2013-01-01,5700,63701.0,481620,0.0,-,4816,NaN,NaN,NaN,not in aggreement,left_only
3,2013-01-01,5700,40265.0,490290,0.0,-,4902,NaN,NaN,NaN,not in aggreement,left_only
4,2013-01-01,5700,324939.0,480431,0.0,-,4804,NaN,NaN,NaN,not in aggreement,left_only


In [108]:
out_file = ".\\data"+ "\\phaseone-tradedata-test.parquet"

pq.write_table(pa.Table.from_pandas(df_phaseone), out_file)

#### This then constructs the benchmark and goal measures

In [109]:
df_phaseone.set_index("time", inplace = True)

In [110]:
grp = df_phaseone.loc["2017"].groupby("high_catagory")

benchmarks = grp.agg({"china_exports": "sum"})

benchmarks.columns = ["2017 Values"]

In [111]:
grp = df_phaseone.loc["2020"].groupby("high_catagory")

current = grp.agg({"china_exports": "sum"})

current.columns = ["2020 Values"]

In [112]:
benchmarks = benchmarks.merge(current, left_index = True, right_index = True)

In [113]:
benchmarks

,2017 Values,2020 Values
high_catagory,,
1. Manufactured Goods,5.015148e+10,2.370067e+10
2. Agriculture,2.096979e+10,6.493618e+09
3. Energy,7.649222e+09,2.925023e+09
not in aggreement,5.026467e+10,1.338490e+10


In [123]:
benchmarks

,2017 Values,2020 Values
high_catagory,,
1. Manufactured Goods,5.015148e+10,1.931454e+10
2. Agriculture,2.096979e+10,5.419944e+09
3. Energy,7.649222e+09,1.973504e+09
not in aggreement,5.026467e+10,1.331000e+10


In [114]:
# These are the goals from the AGREEMENT

benchmarks["Goals"] = 32900000000

benchmarks.iloc[1,2] = 12500000000

benchmarks.iloc[2,2] = 18500000000

benchmarks.iloc[3,2] = np.nan

Then the ``phaseone-goals.parquet`` file is the data file used to create the bar graph.

In [115]:
out_file = ".\\data"+ "\\phaseone-goals-test.parquet"

pq.write_table(pa.Table.from_pandas(benchmarks), out_file)

In [116]:
benchmarks

,2017 Values,2020 Values,Goals
high_catagory,,,
1. Manufactured Goods,5.015148e+10,2.370067e+10,3.290000e+10
2. Agriculture,2.096979e+10,6.493618e+09,1.250000e+10
3. Energy,7.649222e+09,2.925023e+09,1.850000e+10
not in aggreement,5.026467e+10,1.338490e+10,NaN


---
#### The code below then performs the projection of the data down to the county level

The first step is to group on NAICS. Note that how this is constructed is only Phase One coverd products are included. 

In [58]:
df_phaseone.loc["2017":]

,CTY_CODE,china_exports,hs6,hs4,description,low_catagory,high_catagory,_merge,naics,naics3
time,,,,,,,,,,
2017-01-01,5700,753280.0,200899,2008,"Fruit, nuts and other edible parts of plants, ...",Other agricultural commodities,2. Agriculture,both,311421,311
2017-01-01,5700,361441.0,151800,1518,"Animal or vegetable fats, oils and their fract...",Other agricultural commodities,2. Agriculture,both,311613,311
2017-01-01,5700,289709.0,152000,1520,"Glycerol (glycerine), whether or not pure; gly...",Other agricultural commodities,2. Agriculture,both,325611,325
2017-01-01,5700,36631.0,152110,1521,"Vegetable waxes (other than triglycerides), be...",Other agricultural commodities,2. Agriculture,both,325998,325
2017-01-01,5700,2285893.0,160239,1602,"Other prepared or preserved meat, meat offal o...",Meat,2. Agriculture,both,311615,311
...,...,...,...,...,...,...,...,...,...,...
2020-05-01,5700,23320264.0,852351,8523,"Discs, tapes, solid-state non-volatile storage...",Electrical equipment and mac.hioery,1. Manufactured Goods,both,334613,334
2020-05-01,5700,341171.0,852352,8523,"Discs, tapes, solid-state non-volatile storage...",Electrical equipment and mac.hioery,1. Manufactured Goods,both,334413,334
2020-05-01,5700,32764.0,852359,8523,"Discs, tapes, solid-state non-volatile storage...",Electrical equipment and mac.hioery,1. Manufactured Goods,both,334613,334


In [59]:
df_phaseone = df_phaseone.loc["2017":]

grp = df_phaseone.groupby(["time","naics3"])

exports_by_naics = grp.agg({"china_exports": "sum"})

exports_by_naics.reset_index(inplace = True)

exports_by_naics.set_index(["naics3"], inplace = True)

##################################################################

grp = df_phaseone[df_phaseone["high_catagory"] != "not in aggreement"].groupby(["time","naics3"])

exports_phaseone = grp.agg({"china_exports": "sum"})

exports_phaseone.reset_index(inplace = True)

exports_phaseone.set_index(["naics3"], inplace = True)

##################################################################

exports_phaseone.rename(mapper = {"china_exports": "phase_one_exports"}, inplace = True, axis = 1)

exports_by_naics = exports_by_naics.merge(exports_phaseone, 
                                          left_on = ["naics3", "time"], right_on = ["naics3", "time"])

In [60]:
exports_by_naics.tail()

,time,china_exports,phase_one_exports
naics3,,,
337,2020-05-01,4748678.0,0.0
339,2020-05-01,357396485.0,308669410.0
910,2020-05-01,121597022.0,4654176.0
930,2020-05-01,10236239.0,4171017.0
990,2020-05-01,101889086.0,284077.0


The following functions do the following: 

1. Grab the BLS QCEW file for 2017

2. Then create the exports per worker measure at the county level. 

In [61]:
def download_bls():

    print("")
    print("**********************************************************************************")
    print("Downloading and processing BLS file")
    print("")

    url = "https://data.bls.gov/cew/data/files/2017/csv/2017_annual_singlefile.zip"
# This will read in the annual, single file. It's big, but has all we want...

    r = requests.get(url) 

# convert bytes to zip file  
    bls_sf = zf.ZipFile(io.BytesIO(r.content)) 
    print('Type of zipfile object:', type(bls_sf))

    clist = ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code',
       'year', 'disclosure_code', 'annual_avg_estabs',
       'annual_avg_emplvl', 'total_annual_wages','avg_annual_pay']

    df = pd.read_csv(bls_sf.open(bls_sf.namelist()[0]), usecols= clist)

########################################################################

    NAICS_county_level = 75 
# This is the code that will select only counties at the 3 digit NAICS level

    df_county = df[df.agglvl_code == NAICS_county_level].copy()

    df_county = df_county[df_county.own_code == 5]
# Only grab private stuff

    df_county = df_county[(df_county.area_fips.str[0:2] != "72") & (df_county.area_fips.str[0:2] != "78")
              & (df_county.area_fips.str[0:2] != "02") & (df_county.area_fips.str[0:2] != "15")]
#Drop puerto rico, alaska, hawaii...this mayb not be doing what I think it is...as it looks like these guys are there
# Does not matter as analysis is performed withthem, drop them when do the map. 

    df_county["sup_ind"] = df_county.industry_code.str[1].astype(int)
# sometimes there are super industries floating around we want to drop them.
# not clear if this matters with the conditioning all ready

    df_county = df_county[df_county["sup_ind"] > 0]

    df_county.area_fips = df_county.area_fips.astype(str)

    df_national = df_county.groupby("industry_code").agg({"annual_avg_emplvl": "sum"})

    df_national.reset_index(inplace = True)

    df_national.rename({"annual_avg_emplvl":"nat_emplvl"}, axis = 1, inplace = True)
    
    return df_county, df_national

In [62]:
def create_trade_weights(df):
    # Takes in the county groupings and will return, for each county, a time series of export
    # exposure, tariffs, and other statistics. 

    new_df = df.merge(df_national[["nat_emplvl",
                                   "industry_code"]],
                                  how = "outer", left_on = "industry_code", right_on = "industry_code")
    # Merge the nation with the county, why, we want to make sure all the naics codes are lined up properly
        
    new_df["emp_wts"] = (new_df.annual_avg_emplvl/new_df.nat_emplvl)
     
    # create the weights...
        
    foo_df = exports_by_naics.merge(new_df[["emp_wts",
                                           "industry_code",
                                          "annual_avg_emplvl"]], left_index = True, right_on = "industry_code")  
    
    # Now each weight is for a NAICS code, we will merge it with the export trade data set, so for all naics, all time...
    # This is a big df whith all trade data and then the county's weights for each naics code
    
    foo_grp = foo_df.groupby("time")
    
    # group by time. 
    
    foo = foo_grp.apply(trade_by_naics)
    
    # Then for each time gropuing, we aggregate across the naics codes according to the weights above.
    
    foo = foo.droplevel(1)
    
    foo["fips"] = df["area_fips"].astype(str).iloc[0]
    
    # some cleaning of the df
    
    foo["total_employment"] = new_df.annual_avg_emplvl.sum()
    
    # get total employment.
    
    return pd.DataFrame(foo)
    

In [63]:
def trade_by_naics(df):
    # Simple function just to test about aggregation 

    china_exp_pc = (1/df["annual_avg_emplvl"].sum())*(df["china_exports"]*df["emp_wts"]).sum()
    
    china_pho_pc = (1/df["annual_avg_emplvl"].sum())*(df["phase_one_exports"]*df["emp_wts"]).sum()
    # the first term multiplies trade by the county's share of national level employment
    # then the outside term divides by number of workers in a county. 
    
    foo = {"china_exp_pc": [china_exp_pc],
           "china_pho_pc": [china_pho_pc],
          "emplvl_2017": df["annual_avg_emplvl"].sum()}

    return pd.DataFrame(foo)

In [64]:
df_county, df_national = download_bls()

#print(df_county.annual_avg_emplvl.sum())

grp = df_county.groupby("area_fips")

trade_county = grp.apply(create_trade_weights)

trade_county["china_exp_pc"] = (trade_county["emplvl_2017"]/
                                    trade_county["total_employment"])*trade_county["china_exp_pc"]

trade_county["china_pho_pc"] = (trade_county["emplvl_2017"]/
                                    trade_county["total_employment"])*trade_county["china_pho_pc"]


**********************************************************************************

Type of zipfile object: <class 'zipfile.ZipFile'>


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [65]:
trade_county.head()

china_exp_pc  china_pho_pc  emplvl_2017   fips  \
area_fips time                                                         
10001     2017-01-01     58.576866     53.597833       2843.0  10001   
          2017-02-01     48.538294     43.914389       2843.0  10001   
          2017-03-01     42.466573     37.635078       2843.0  10001   
          2017-04-01     33.938510     29.675054       2843.0  10001   
          2017-05-01     34.109425     30.018439       2843.0  10001   

                      total_employment  
area_fips time                          
10001     2017-01-01           29514.0  
          2017-02-01           29514.0  
          2017-03-01           29514.0  
          2017-04-01           29514.0  
          2017-05-01           29514.0

In [66]:
trade_county.head()

china_exp_pc  china_pho_pc  emplvl_2017   fips  \
area_fips time                                                         
10001     2017-01-01     58.576866     53.597833       2843.0  10001   
          2017-02-01     48.538294     43.914389       2843.0  10001   
          2017-03-01     42.466573     37.635078       2843.0  10001   
          2017-04-01     33.938510     29.675054       2843.0  10001   
          2017-05-01     34.109425     30.018439       2843.0  10001   

                      total_employment  
area_fips time                          
10001     2017-01-01           29514.0  
          2017-02-01           29514.0  
          2017-03-01           29514.0  
          2017-04-01           29514.0  
          2017-05-01           29514.0

And we are set. The only final part is to add in some information from the census. 

In [67]:
my_api_key = '34e40301bda77077e24c859c6c6c0b721ad73fc7'
# This is my api_key

c = Census(my_api_key)
# This will create an object c which has methods associated with it.
# We will see  these below.

type(c) 
# Per the discussion below, try c.tab and see the options. 

code = ("NAME","B01001_001E","B19013_001E") # Same Codes:

county_2017 = pd.DataFrame(c.acs5.get(code, 
                                         {'for': 'county:*'}, year=2017))
                                         # Same deal, but we specify county then the wild card
                                         # On the example page, there are ways do do this, only by state
        
county_2017 = county_2017.rename(columns = {"B01001_001E":"2017_population", "B19013_001E":"2017_income"})

county_2017["GEOFIPS"] = (county_2017["state"] + county_2017["county"]).astype(int)

county_2017["2017_population"] = county_2017["2017_population"].astype(float)

county_2017["2017_income"] = county_2017["2017_income"].astype(float)

county_2017.set_index(["GEOFIPS"], inplace = True)

In [68]:
trade_county.reset_index(inplace = True)

trade_county["int_area_fips"] = trade_county["area_fips"].astype(int)

trade_county = trade_county.merge(county_2017[["2017_income","2017_population"]],
                                  left_on = "int_area_fips", right_index = True, how = "left")

#trade_employ.drop(labels = "index", axis = 1, inplace = True)

trade_county.set_index(["area_fips", "time"],inplace = True)

In [69]:
trade_county.head()

china_exp_pc  china_pho_pc  emplvl_2017   fips  \
area_fips time                                                         
10001     2017-01-01     58.576866     53.597833       2843.0  10001   
          2017-02-01     48.538294     43.914389       2843.0  10001   
          2017-03-01     42.466573     37.635078       2843.0  10001   
          2017-04-01     33.938510     29.675054       2843.0  10001   
          2017-05-01     34.109425     30.018439       2843.0  10001   

                      total_employment  int_area_fips  2017_income  \
area_fips time                                                       
10001     2017-01-01           29514.0          10001      57647.0   
          2017-02-01           29514.0          10001      57647.0   
          2017-03-01           29514.0          10001      57647.0   
          2017-04-01           29514.0          10001      57647.0   
          2017-05-01           29514.0          10001      57647.0   

                      2017_population  
area_fips time                         
10001     2017-01-01         173145.0  
          2017-02-01         173145.0  
          2017-03-01         173145.0  
          2017-04-01         173145.0  
          2017-05-01         173145.0

In [70]:
file_path =  ".\\data"+ "\\phase_one_county.parquet"

pq.write_table(pa.Table.from_pandas(trade_county), file_path)

Then the ``phase_one_county.parquet`` file is the main file used in the ``phase-one-map.ipynb`` notebook.